In [ ]:
!pip install dostoevsky

In [ ]:
!pip install transformers

In [7]:
!python3 -m dostoevsky download fasttext-social-network-model

In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import math
import pandas as pd
import torch
from tqdm import tqdm
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [14]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [65]:
DIRID = 'roscomsvoboda'
PUBID = "roscomsv"

In [67]:
comments = pd.read_parquet(f"/content/drive/My Drive/{DIRID}/comments_{PUBID}.parquet")

In [54]:
comm_filt = comments[comments.text != '']
comm_filt = comm_filt[comm_filt.text.str.contains('[А-яа-я]')]
comm_filt.index = pd.RangeIndex(len(comm_filt.index))
len(comm_filt)

77839

In [55]:
texts = comm_filt.text
results = model.predict(texts, k=2)

In [56]:
res_labels = []
for kvp in results:
  maxkey = max(kvp, key=kvp.get)
  if (maxkey == 'skip'):
    maxkey = min(kvp, key=kvp.get)
  res_labels.append(maxkey)

In [57]:
len(res_labels)

77839

In [61]:
res_labels[3]

'negative'

In [62]:
comm_filt['prediction'] = res_labels

In [63]:
comm_filt.head()

,date,from_id,id,owner_id,post_id,text,prediction
0,1653989748,-42716524,111376,-45023092,111375,"Под предлогом борьбы с цензурой , поставить св...",neutral
1,1653994537,14338960,111377,-45023092,111375,А с каких пор провластная точка зрения априори...,negative
2,1654003739,20520693,111386,-45023092,111375,"""Родительский контроль направлен на то, чтобы ...",neutral
3,1656244206,162855206,112248,-45023092,111375,"Мало нам цензуры государства, теперь имеем ещё...",negative
4,1656089020,538546283,112230,-45023092,112229,Если иконку Фэйсбука кто - нибудь в посте или ...,neutral


In [68]:
comm_filt.to_parquet(f"/content/drive/My Drive/{DIRID}/users_friends_sent_{PUBID}.parquet")